### Setup

In [3]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import style
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
style.use("ggplot")

### Load Dataset and Rndomization

In [6]:
california_housing_dataframe = pd.read_csv("https://storage.googleapis.com/mledu-datasets/california_housing_train.csv", sep=",")
california_housing_dataframe = california_housing_dataframe.reindex(np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207300.9
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,115983.8
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,14999.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119400.0
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180400.0
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265000.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500001.0


### Preprocess Data

In [7]:
def preprocess_features(california_housing_dataframe):
    selected_features = california_housing_dataframe[
        ["latitude",
         "longitude",
         "housing_median_age",
         "total_rooms",
         "total_bedrooms",
         "population",
         "households",
         "median_income"]
    ]
    preprocessed_features = selected_features.copy()
    preprocessed_features["rooms_per_person"] = (california_housing_dataframe["total_rooms"]/california_housing_dataframe["population"])
    return preprocessed_features

def preprocess_targets(california_housing_dataframe):
    output_targets = pd.DataFrame()
    output_targets["median_house_value"] = (california_housing_dataframe["median_house_value"] / 1000.0)
    return output_targets

### Training set and Validation set

In [12]:
training_examples = preprocess_features(california_housing_dataframe.head(12000))
training_targets = preprocess_targets(california_housing_dataframe.head(12000))

validation_examples = preprocess_features(california_housing_dataframe.tail(5000))
validation_targets = preprocess_targets(california_housing_dataframe.tail(5000))

print("Training examples Summary:")
display.display(training_examples.describe())
print("Validation examples Summary:")
display.display(validation_examples.describe())

print("Training targets Summary:")
display.display(training_targets.describe())
print("Validation targets Summary:")
display.display(validation_targets.describe())

Training examples Summary:


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0,12000.0
mean,35.6,-119.6,28.6,2650.6,541.0,1433.5,502.6,3.9,2.0
std,2.1,2.0,12.6,2149.6,420.2,1159.4,384.9,1.9,1.2
min,32.5,-124.3,1.0,2.0,1.0,3.0,1.0,0.5,0.0
25%,33.9,-121.8,18.0,1469.0,298.0,794.0,282.0,2.6,1.5
50%,34.2,-118.5,29.0,2142.0,434.0,1170.0,410.0,3.5,1.9
75%,37.7,-118.0,37.0,3157.2,652.0,1728.2,607.0,4.7,2.3
max,42.0,-114.3,52.0,32627.0,6445.0,35682.0,6082.0,15.0,55.2


Validation examples Summary:


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_person
count,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0
mean,35.6,-119.6,28.7,2627.1,535.7,1420.1,498.0,3.9,2.0
std,2.1,2.0,12.6,2251.2,424.7,1119.8,383.6,1.9,1.2
min,32.5,-124.3,2.0,11.0,4.0,8.0,2.0,0.5,0.1
25%,33.9,-121.8,18.0,1438.8,295.0,779.8,280.8,2.6,1.5
50%,34.2,-118.5,29.0,2103.5,432.0,1159.5,405.5,3.5,1.9
75%,37.7,-118.0,37.0,3135.0,641.0,1700.0,599.0,4.8,2.3
max,41.9,-114.6,52.0,37937.0,5471.0,16122.0,5189.0,15.0,52.0


Training targets Summary:


,median_house_value
count,12000.0
mean,206.7
std,115.4
min,15.0
25%,119.3
50%,179.5
75%,264.4
max,500.0


Validation targets Summary:


,median_house_value
count,5000.0
mean,208.9
std,117.4
min,17.5
25%,120.0
50%,181.5
75%,265.9
max,500.0
